# Finding scope 3 emissions of a Pharma company.
- Scope 3 category is downstream transportation and distribution
- Data taken from data.world
- Distance-based methology will be used for calculations
    - According to GHG "If sub-contractor fuel data cannot be easily obtained in order to use the fuel-based method, then the distance-based method should be used."

# Data points required for calculations of scope 3
1. Total distance in KM 
    - Coordinates of places
2. Size of packages
3. Mode of Transport
4. Emission Factors
5. Calculation formulaes
    1. Emissions from road transport: = ∑ (mass of goods purchased (tonnes) × distance travelled in transport leg × emission factor of transport mode or vehicle type (kg CO2e/tonne-km)
    2. emissions from air transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))
    3. emissions from sea transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))

### Steps I took
1. Searching for dataset
2. Understanding the dataset and making calculative assumptions
    - Assumptions made should have proper justification.
3. Listing down the data points required for calculations
4. Listing down the calculation formulaes
5. Filtering out the data columns required from main data
6. Exploratory Data Analysis
7. Choosing a single year for calculations


## Data columns
1. Country - Destination location
2. Mode - Mode of transport (Air, Ocean, Road)
3. Weight - Kilogram of weight
4. Delivery Date - Date of delivery
5. Manufacturing Site - Source location

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Collecting data
raw_df = pd.read_csv("./Data/SCMS_Delivery_History_Dataset_20150929.csv")

In [90]:
# Filtering out required columns from the main data.

filtered_df = raw_df[['ID','Country','Shipment Mode','Manufacturing Site','Weight (Kilograms)','Delivery Recorded Date','Item Description']].copy()
filtered_df.rename(columns={"Shipment Mode":"Mode","Weight (Kilograms)":"Weight","Delivery Recorded Date":"Delivery_Date",'Manufacturing Site':'Manufacturing_site'},inplace=True)

In [91]:
# Converting Delivery Date to datetime datatype
filtered_df.loc[:,'Delivery_Date'] = pd.to_datetime(filtered_df['Delivery_Date'])

# Exploratory Data Analysis

In [92]:
# Checking total number of data points for each year
filtered_df['Delivery_Date'].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      65
2007     670
2008    1109
2009    1192
2010    1176
2011    1049
2012    1250
2013    1205
2014    1599
2015    1009
Name: Delivery_Date, dtype: int64

In [93]:
# Check for null values
filtered_df.isna().sum()

ID                      0
Country                 0
Mode                  360
Manufacturing_site      0
Weight                  0
Delivery_Date           0
Item Description        0
dtype: int64

Null values found in the mode of transport. 

In [94]:
# Checking Mode column's null values

# filtered_df[filtered_df['Mode'].isna()]

# Counting the number of null values per year
filtered_df[filtered_df['Mode'].isna()].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      2
2007    264
2008     94
dtype: int64

Since mode of transport is not available for these years (2006,2007,2008) I will not consider these years for our base year.
Even though year 2006 has only 2 NaN rows for Mode, the number of data points are only 65. Thus not enough to consider as base year as compared to other years.

In [95]:
# Checking Weight column values
filtered_df[filtered_df['Weight'].str.isnumeric() == False]

,ID,Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description
8,46,Nigeria,Air,"Aurobindo Unit III, India",See ASN-93 (ID#:1281),2006-12-07,"Stavudine 30mg, capsules, 60 Caps"
12,62,Nigeria,Air,"EY Laboratories, USA",Weight Captured Separately,2007-01-10,"HIV 1/2, InstantChek HIV 1+2 Kit, 100 Tests"
15,68,Zimbabwe,Air,"BMS Meymac, France",Weight Captured Separately,2007-03-19,"#102198**Didanosine 200mg [Videx], tablets, 60..."
16,69,Nigeria,NaN,ABBVIE GmbH & Co.KG Wiesbaden,Weight Captured Separately,2007-05-07,"HIV 1/2, Determine Complete HIV Kit, 100 Tests"
31,262,South Africa,NaN,GSK Mississauga (Canada),Weight Captured Separately,2008-01-29,"Zidovudine 10mg/ml [Retrovir], oral solution, ..."
...,...,...,...,...,...,...,...
10318,86817,Zimbabwe,Truck,"Cipla, Goa, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10319,86818,Zimbabwe,Truck,"Mylan, H-12 & H-13, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10320,86819,C�te d'Ivoire,Truck,Hetero Unit III Hyderabad IN,See DN-4313 (ID#:83921),2015-08-07,"Lamivudine/Zidovudine 150/300mg, tablets, 60 Tabs"
10321,86821,Zambia,Truck,Cipla Ltd A-42 MIDC Mahar. IN,Weight Captured Separately,2015-09-03,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...


As you can see above the weight column has non-numeric values 'Weight Captured Separately', thus the values for these rows are not available. While values like 'See ASN-93 (ID#:1281)' can be found by mapping to that particular ID and getting the weight

In [96]:
# Counting the number of 'Weight Captured Separately' values in weight columns per year.

filtered_df[(filtered_df['Weight'].str.isnumeric() == False) & (filtered_df['Weight'] == 'Weight Captured Separately')].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      6
2007     28
2008    233
2009    262
2010     67
2011     56
2012     33
2013     80
2014    386
2015    356
dtype: int64

Seeing that 2012 has least number of 'Weight Captured Separately' values and it has good number of data points i.e. 1250. We can consider it as our base year.

# Data Cleaning

## Fixing Weight column

In [97]:
# Filtering out 'Weight Captured Separately' rows from Final dataset
final_df = filtered_df[(filtered_df['Delivery_Date'].dt.year == 2012) & (filtered_df['Weight'] != 'Weight Captured Separately')].copy()
final_df.reset_index(inplace = True, drop = True)

In [98]:
# Function to resolve "See DN-2947 (ID#:83642)" given in weight column and adding the weight

def mapWeights(weight):
    try:
        if weight.isnumeric() == False:
            ID = weight[-6:-1]
            weight_returned = filtered_df[filtered_df['ID'] == int(ID)]['Weight'].iloc[0]
            if weight_returned == 'Weight Captured Separately':
                return None
#             print(f'{ID} -- {weight_returned}')
            return float(weight_returned)
        return float(weight)
    except Exception as e:
        print(f'Error == {e} \n {weight[-6:-1]} --- {filtered_df[filtered_df["ID"] == int(weight[-6:-1])]["Weight"].iloc[0]}')

In [99]:
weights = final_df['Weight'].apply(mapWeights)

In [100]:
final_df.loc[:,'Weight'] = weights

In [101]:
# Converting Weights from KG to Tonnes
# 1 kilogram = 0.001 tonne

final_df['Weight'] = final_df['Weight'] * 0.001

In [102]:
final_df

,ID,Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description
0,12973,Haiti,Air,"Cipla, Goa, India",0.021,2012-06-12,"Lamivudine/Nevirapine/Stavudine 30/50/6mg, dis..."
1,13016,Burundi,Air,Chembio Diagnostics Sys. Inc.,0.034,2012-06-01,"HIV 1/2, Stat-Pak Dipstick Assay Kit, 30 Tests"
2,13020,Haiti,Air,Premier Medical Corporation,0.001,2012-01-30,"Malaria Antigen P.f Kit, 30 x 1 Test"
3,13311,C�te d'Ivoire,Air,"Alere Medical Co., Ltd.",0.957,2012-03-29,"HIV 1/2, Determine Complete HIV Kit, 100 Tests"
4,13608,Uganda,Air,"Janssen-Cilag, Latina, IT",0.043,2012-05-21,"Etravirine 100mg [Intelence], tablets, 120 Tabs"
...,...,...,...,...,...,...,...
1212,86529,Uganda,Truck,Hetero Unit III Hyderabad IN,0.001,2012-11-20,"Zidovudine 300mg, tablets, 60 Tabs"
1213,86530,Uganda,Air,"Aurobindo Unit III, India",2.034,2012-11-28,"Lopinavir/Ritonavir 200/50mg, tablets, 120 Tabs"
1214,86532,Nigeria,Air,Mylan (formerly Matrix) Nashik,50.634,2012-12-04,Lamivudine/Nevirapine/Zidovudine 150/200/300mg...
1215,86533,C�te d'Ivoire,Truck,Mylan (formerly Matrix) Nashik,4.409,2012-12-06,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...


## Getting Location coordinates of Source Location and Destination Location

### If Mode of Transport is,
1. Air and Truck, then we get the Airport coordinates of the Capital City of the Country given
2. Ocean, then we get the coordinates of the main port of the country.